In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession,Row

In [3]:
spark = SparkSession.builder.appName('recreate2').getOrCreate()

In [5]:
# This is one direct way, another is to first create RDD then map it to the RDD of Row object and then convert it to
# the dataframe.

# df = spark.read.csv \
#     ('file:///Users/hdagar3/Documents/Spark_Things/Spark_Course_Files_FraneKane/fakefriends.csv',header=False)

In [6]:
# print(df)
# print(type(df))

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string]
<class 'pyspark.sql.dataframe.DataFrame'>


In [7]:
# print(df.show())

+---+--------+---+---+
|_c0|     _c1|_c2|_c3|
+---+--------+---+---+
|  0|    Will| 33|385|
|  1|Jean-Luc| 26|  2|
|  2|    Hugh| 55|221|
|  3|  Deanna| 40|465|
|  4|   Quark| 68| 21|
|  5|  Weyoun| 59|318|
|  6|  Gowron| 37|220|
|  7|    Will| 54|307|
|  8|  Jadzia| 38|380|
|  9|    Hugh| 27|181|
| 10|     Odo| 53|191|
| 11|     Ben| 57|372|
| 12|   Keiko| 54|253|
| 13|Jean-Luc| 56|444|
| 14|    Hugh| 43| 49|
| 15|     Rom| 36| 49|
| 16|  Weyoun| 22|323|
| 17|     Odo| 35| 13|
| 18|Jean-Luc| 45|455|
| 19|  Geordi| 60|246|
+---+--------+---+---+
only showing top 20 rows

None


In [9]:
# df = df.withColumnRenamed("_c0","Id")
# df = df.withColumnRenamed("_c1","Name")
# df = df.withColumnRenamed("_c2","Age")
# df = df.withColumnRenamed("_c3","No_of_friends")
# nothing is inplace, hence did it like this

In [10]:
# print(df.columns)

['Id', 'Name', 'Age', 'No_of_friends']


In [11]:
# df.show()

+---+--------+---+-------------+
| Id|    Name|Age|No_of_friends|
+---+--------+---+-------------+
|  0|    Will| 33|          385|
|  1|Jean-Luc| 26|            2|
|  2|    Hugh| 55|          221|
|  3|  Deanna| 40|          465|
|  4|   Quark| 68|           21|
|  5|  Weyoun| 59|          318|
|  6|  Gowron| 37|          220|
|  7|    Will| 54|          307|
|  8|  Jadzia| 38|          380|
|  9|    Hugh| 27|          181|
| 10|     Odo| 53|          191|
| 11|     Ben| 57|          372|
| 12|   Keiko| 54|          253|
| 13|Jean-Luc| 56|          444|
| 14|    Hugh| 43|           49|
| 15|     Rom| 36|           49|
| 16|  Weyoun| 22|          323|
| 17|     Odo| 35|           13|
| 18|Jean-Luc| 45|          455|
| 19|  Geordi| 60|          246|
+---+--------+---+-------------+
only showing top 20 rows



In [5]:
sc = spark.sparkContext
print(sc)

<SparkContext master=local[*] appName=recreate2>


In [6]:
rdd = sc.textFile('file:///Users/hdagar3/Documents/Spark_Things/Spark_Course_Files_FraneKane/fakefriends.csv')

In [7]:
print(rdd.take(3))

['0,Will,33,385', '1,Jean-Luc,26,2', '2,Hugh,55,221']


In [8]:
def split_function(line):
    info = line.split(",")
    age = int(info[2])
    friends = int(info[3])
    return Row(age = age, no_of_friends=friends)

In [9]:
another_rdd = rdd.map(split_function)

In [11]:
print(another_rdd.top(5))

[Row(age=69, no_of_friends=491), Row(age=69, no_of_friends=470), Row(age=69, no_of_friends=431), Row(age=69, no_of_friends=361), Row(age=69, no_of_friends=236)]


In [12]:
df = spark.createDataFrame(another_rdd).cache()

In [13]:
print(df.show())

+---+-------------+
|age|no_of_friends|
+---+-------------+
| 33|          385|
| 26|            2|
| 55|          221|
| 40|          465|
| 68|           21|
| 59|          318|
| 37|          220|
| 54|          307|
| 38|          380|
| 27|          181|
| 53|          191|
| 57|          372|
| 54|          253|
| 56|          444|
| 43|           49|
| 36|           49|
| 22|          323|
| 35|           13|
| 45|          455|
| 60|          246|
+---+-------------+
only showing top 20 rows

None


In [14]:
grouped_obj = df.groupBy('age')

In [17]:
grouped_obj.avg().orderBy('age').select(['age','avg(no_of_friends)']).show()

+---+------------------+
|age|avg(no_of_friends)|
+---+------------------+
| 18|           343.375|
| 19|213.27272727272728|
| 20|             165.0|
| 21|           350.875|
| 22|206.42857142857142|
| 23|             246.3|
| 24|             233.8|
| 25|197.45454545454547|
| 26|242.05882352941177|
| 27|           228.125|
| 28|             209.1|
| 29|215.91666666666666|
| 30| 235.8181818181818|
| 31|            267.25|
| 32| 207.9090909090909|
| 33| 325.3333333333333|
| 34|             245.5|
| 35|           211.625|
| 36|             246.6|
| 37|249.33333333333334|
+---+------------------+
only showing top 20 rows



In [21]:
# OR
grouped_obj.agg({'no_of_friends':'avg'}).orderBy('age').show()
# grouped_obj.agg({'no_of_friends':'mean'}).orderBy('age').show()

+---+------------------+
|age|avg(no_of_friends)|
+---+------------------+
| 18|           343.375|
| 19|213.27272727272728|
| 20|             165.0|
| 21|           350.875|
| 22|206.42857142857142|
| 23|             246.3|
| 24|             233.8|
| 25|197.45454545454547|
| 26|242.05882352941177|
| 27|           228.125|
| 28|             209.1|
| 29|215.91666666666666|
| 30| 235.8181818181818|
| 31|            267.25|
| 32| 207.9090909090909|
| 33| 325.3333333333333|
| 34|             245.5|
| 35|           211.625|
| 36|             246.6|
| 37|249.33333333333334|
+---+------------------+
only showing top 20 rows



In [ ]:
# END